<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=05a11462013f574e08a08d07fcfce0d67efe8724be04c7eee672bbb40e13e149
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-10 09:21:34
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  11.04 L
Current:  1.41 C
-------------------
Today PnL: 62.41 K (0.44%)
Current PnL: -18.24 L (-12.27%)
CY Booked + Current PnL: -7.21 L (-4.85%)
-------------------
Total profit:  1.89 L
Total loss:  -20.13 L
-------------------
Total Booked + Current PnL: 19.77 L (16.25%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.82%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 77.88 L (55.24%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 9.22%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.53,60.0,H-LC,11.15,172788.0,16611.0,9486.0,0.36,10.64,5.49,16.71,22.0,1.75,1.33,29.93,X40N,BTT,HEALTHCARE
77,TTKPRESTIG,770.00,105.34,61.0,M-SC,5.58,88425.0,-12352.0,12441.0,-0.55,-12.26,14.07,0.09,245.0,-0.99,0.68,16.17,OX40N,NTT,DURABLES
62,SBIN,863.00,-13.25,53.0,M-LC,4.25,215710.0,14230.0,12986.0,0.64,7.06,6.02,13.51,60.0,1.10,1.66,20.55,XY25,NTT,BANKS
41,ITC,452.00,-37.92,51.0,H-LC,3.29,200441.0,303.0,20585.0,-0.04,0.15,10.27,10.44,4.0,0.01,1.54,5.72,X40,NTT,FMCG
49,NATIONALUM,247.44,-44.49,66.0,H-MC,2.82,110542.0,10038.0,20605.0,0.94,9.99,18.64,30.49,79.0,0.49,0.85,45.63,MH,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,41.14,60.0,M-MC,7.06,223051.0,-1911.0,169563.0,0.06,-0.85,76.02,74.53,192.0,-0.01,1.72,31.13,XY24,BTT,STEEL
35,ICICIPRULI,790.00,-21.22,40.0,H-MC,2.01,135600.0,-188.0,42945.0,0.31,-0.14,31.67,31.48,107.0,-0.00,1.04,11.96,X40,ATH,INSURANCE
30,HAVELLS,2069.16,0.20,62.0,H-MC,1.74,248352.0,621.0,74431.0,0.31,0.25,29.97,30.30,92.0,0.01,1.91,13.58,X40,ATH,ELECTRICAL
34,ICICIGI,2260.25,-20.58,42.0,H-MC,3.48,136604.0,471.0,30654.0,0.60,0.35,22.44,22.86,91.0,0.02,1.05,13.00,X40,ATH,INSURANCE
51,PGHH,17973.08,-30.68,53.0,H-MC,4.69,200730.0,-90.0,68870.0,-0.05,-0.04,34.31,34.25,80.0,-0.00,1.54,5.15,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-15.07,42.0,H-LC,6.37,305602.0,-10240.0,122913.0,0.95,-3.24,40.22,35.67,5.0,-0.08,2.35,10.16,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.23,44.0,H-LC,13.77,218365.0,959.0,52539.0,1.30,0.44,24.06,24.61,48.0,0.02,1.68,55.50,XY24,BTT,FINANCE
31,HCLTECH,1943.91,-0.22,44.0,H-LC,10.21,227493.0,-14431.0,77712.0,1.53,-5.97,34.16,26.15,8.0,-0.19,1.75,7.50,X40,ATH,IT
56,RELIANCE,1533.00,-14.04,47.0,H-LC,6.50,215389.0,5023.0,23757.0,0.33,2.39,11.03,13.68,37.0,0.21,1.66,19.31,XY25,NTT,REFINERIES
80,UNITDSPR,1644.00,-13.94,47.0,H-LC,8.24,173026.0,-8733.0,43983.0,0.49,-4.80,25.42,19.39,86.0,-0.20,1.33,2.95,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-4.05,58.0,H-LC,1.35,237960.0,-8191.0,87403.0,1.98,-3.33,36.73,32.18,16.0,-0.09,1.83,31.52,X200,ATH,IT
50,NESTLEIND,1377.00,-8.60,59.0,H-LC,2.35,281315.0,15889.0,40903.0,1.22,5.99,14.54,21.40,11.0,0.39,2.16,13.03,XY25,NTT,FMCG
41,ITC,452.00,-37.92,51.0,H-LC,3.29,200441.0,303.0,20585.0,-0.04,0.15,10.27,10.44,4.0,0.01,1.54,5.72,X40,NTT,FMCG
20,CIPLA,1795.00,-20.12,51.0,H-LC,5.93,213583.0,9083.0,32336.0,0.48,4.44,15.14,20.25,10.0,0.28,1.64,13.56,X40N,BTT,PHARMA
83,VBL,671.64,-15.07,42.0,H-LC,6.37,305602.0,-10240.0,122913.0,0.95,-3.24,40.22,35.67,5.0,-0.08,2.35,10.16,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TRIDENT,48.00,-7.75,57.0,M-SC,6.03,71754.0,-20500.0,46805.0,3.34,-22.22,65.23,28.51,224.0,-0.44,0.55,22.78,XR,NTT,TEXTILES
72,TATAELXSI,9161.00,-20.18,54.0,H-MC,8.32,79520.0,-23132.0,48738.0,2.28,-22.53,61.29,24.94,98.0,-0.47,0.61,20.88,OX40N,NTT,IT
22,COFFEEDAY,80.00,-37.63,61.0,L-SC,42.30,91680.0,-21869.0,61921.0,2.27,-19.26,67.54,35.27,268.0,-0.35,0.71,124.39,XR,NTT,HOTELS
16,BSOFT,836.99,-13.25,52.0,H-SC,16.45,66911.0,-43138.0,79564.0,2.27,-39.20,118.91,33.10,131.0,-0.54,0.51,11.18,XR,ATH,IT
47,LTIM,7230.20,-4.05,58.0,H-LC,1.35,237960.0,-8191.0,87403.0,1.98,-3.33,36.73,32.18,16.0,-0.09,1.83,31.52,X200,ATH,IT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VALIANTORG,838.00,-332.69,60.0,H-SC,8.71,98800.0,-34486.0,119084.0,-2.49,-25.87,120.53,63.47,139.0,-0.29,0.76,65.61,XR,NTT,CHEMICALS
46,LAOPALA,464.00,118.16,53.0,H-SC,3.03,69260.0,-31586.0,57410.0,-1.31,-31.32,82.89,25.61,142.0,-0.55,0.53,28.41,AR,NTT,CERAMICS
85,WHIRLPOOL,2270.00,-39.65,59.0,M-SC,2.51,101955.0,10456.0,68300.0,-0.86,11.43,66.99,86.07,223.0,0.15,0.78,48.06,XR,NTT,DURABLES
64,SHALBY,327.00,1070.65,72.0,M-SC,2.36,159113.0,-22030.0,67496.0,-0.76,-12.16,42.42,25.10,235.0,-0.33,1.22,27.51,XY24,NTT,HEALTHCARE
36,INDIAMART,4850.92,-51.47,47.0,H-SC,12.10,135563.0,12227.0,121532.0,-0.66,9.91,89.65,108.45,119.0,0.10,1.04,34.48,AR,ATH,MISC


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,103.03,69.0,H-SC,12.78,128439.0,-14040.0,33561.0,0.21,-9.85,26.13,13.70,163.0,-0.42,0.99,51.59,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,105.34,61.0,M-SC,5.58,88425.0,-12352.0,12441.0,-0.55,-12.26,14.07,0.09,245.0,-0.99,0.68,16.17,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.42,63.0,H-SC,2.39,224001.0,-45666.0,82007.0,0.09,-16.93,36.61,13.47,138.0,-0.56,1.72,13.30,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,113.73,48.0,M-SC,3.03,140108.0,-34451.0,34481.0,0.20,-19.74,24.61,0.02,221.0,-1.00,1.08,21.87,OX40N,NTT,PAINTS
19,CERA,9475.0,-20.05,45.0,H-SC,2.29,114030.0,-29773.0,56525.0,-0.15,-20.70,49.57,18.60,149.0,-0.53,0.88,25.84,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-27.08,50.0,H-LC,13.96,291203.0,-54753.0,178624.0,1.59,-15.83,61.34,35.80,1.0,-0.31,2.24,3.16,X40,BTT,IT
39,INFY,2275.00,-16.73,56.0,H-LC,10.14,324314.0,11426.0,160276.0,1.22,3.65,49.42,54.87,3.0,0.07,2.50,10.54,X40,BTT,IT
41,ITC,452.00,-37.92,51.0,H-LC,3.29,200441.0,303.0,20585.0,-0.04,0.15,10.27,10.44,4.0,0.01,1.54,5.72,X40,NTT,FMCG
83,VBL,671.64,-15.07,42.0,H-LC,6.37,305602.0,-10240.0,122913.0,0.95,-3.24,40.22,35.67,5.0,-0.08,2.35,10.16,X40N,ATH,FMCG
1,ABB,7934.00,-40.64,49.0,H-LC,12.54,248026.0,-13593.0,132818.0,0.69,-5.20,53.55,45.57,7.0,-0.10,1.91,5.54,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,2326.38,9.63,54.0,H-MC,3.55,189370.0,24700.0,43271.0,0.36,15.00,22.85,41.28,89.0,0.57,1.46,24.18,X40N,ATH,AC
4,ANGELONE,3033.00,9.18,42.0,H-SC,6.36,193200.0,2194.0,55506.0,0.86,1.15,28.73,30.21,157.0,0.04,1.49,22.27,X40N,NTT,FINANCE
84,VOLTAS,1918.49,1.02,59.0,H-MC,3.29,211875.0,20133.0,75894.0,0.01,10.50,35.82,50.08,99.0,0.27,1.63,18.01,XY25,ATH,AC
30,HAVELLS,2069.16,0.20,62.0,H-MC,1.74,248352.0,621.0,74431.0,0.31,0.25,29.97,30.30,92.0,0.01,1.91,13.58,X40,ATH,ELECTRICAL
24,DABUR,735.00,-2.22,62.0,H-MC,5.58,212862.0,15834.0,73778.0,0.28,8.04,34.66,45.49,102.0,0.21,1.64,22.12,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-44.49,66.0,H-MC,2.82,110542.0,10038.0,20605.0,0.94,9.99,18.64,30.49,79.0,0.49,0.85,45.63,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-39.65,59.0,M-SC,2.51,101955.0,10456.0,68300.0,-0.86,11.43,66.99,86.07,223.0,0.15,0.78,48.06,XR,NTT,DURABLES
36,INDIAMART,4850.92,-51.47,47.0,H-SC,12.10,135563.0,12227.0,121532.0,-0.66,9.91,89.65,108.45,119.0,0.10,1.04,34.48,AR,ATH,MISC
86,WIPRO,420.00,-10.62,57.0,M-LC,6.53,157257.0,6312.0,103145.0,1.80,4.18,65.59,72.51,53.0,0.06,1.21,10.46,XR,NTT,IT
12,BANKINDIA,190.00,-31.68,52.0,H-MC,5.46,175071.0,-4737.0,117158.0,0.94,-2.63,66.92,62.52,88.0,-0.04,1.35,30.55,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6711.11,51.0,L-SC,6.08,74715.0,-19095.0,95717.0,0.00,-20.36,128.11,81.67,269.0,-0.20,0.57,44.69,XR,NTT,CERAMICS
3,ALKYLAMINE,4546.37,-5.68,42.0,H-SC,14.66,87767.0,-13196.0,107725.0,0.56,-13.07,122.74,93.63,148.0,-0.12,0.68,30.73,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-37.63,61.0,L-SC,42.30,91680.0,-21869.0,61921.0,2.27,-19.26,67.54,35.27,268.0,-0.35,0.71,124.39,XR,NTT,HOTELS
48,MASFIN,398.61,-15.54,61.0,H-SC,13.03,95820.0,-2160.0,23763.0,0.09,-2.20,24.80,22.05,152.0,-0.09,0.74,39.28,XR,ATH,FINANCE
82,VALIANTORG,838.00,-332.69,60.0,H-SC,8.71,98800.0,-34486.0,119084.0,-2.49,-25.87,120.53,63.47,139.0,-0.29,0.76,65.61,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-37.63,61.0,L-SC,42.30,91680.0,-21869.0,61921.0,2.27,-19.26,67.54,35.27,268.0,-0.35,0.71,124.39,XR,NTT,HOTELS
60,SAMMAANCAP,326.0,-171.59,63.0,M-SC,3.96,81960.0,-20250.0,113638.0,0.21,-19.81,138.65,91.37,208.0,-0.18,0.63,33.05,XY25,NTT,FINANCE
8,ATULAUTO,844.0,3694.59,62.0,M-SC,7.61,118295.0,-29073.0,80890.0,-0.66,-19.73,68.38,35.16,236.0,-0.36,0.91,20.83,XY24,NTT,AUTO
75,TITAGARH,1548.0,-4.73,62.0,H-SC,7.19,156284.0,-35731.0,114619.0,1.03,-18.61,73.34,41.08,158.0,-0.31,1.20,32.43,XY24,NTT,ENGINEERING
55,RELAXO,1176.0,-31.75,60.0,H-SC,2.86,63360.0,-52178.0,87171.0,0.01,-45.16,137.58,30.28,136.0,-0.60,0.49,24.57,X40N,NTT,FOOTWEAR


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.36
1,25,44.54
2,50,75.07


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.79
LC    35.43
MC    23.77
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.65
X40      16.08
X40N     13.17
XY25     10.30
XR        9.52
AR        8.31
OX40N     7.76
X200      1.83
MH        1.79
X5K       1.46
SR        1.12
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.87
H-SC    24.94
H-MC    20.68
M-SC    14.17
M-LC     7.45
M-MC     2.76
L-SC     1.68
L-LC     1.11
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,12.90,-16.24,75.10
FMCG,12.33,-3.55,38.65
FINANCE,9.08,-18.41,60.99
BANKS,7.70,-13.90,62.27
PAINTS,6.18,-14.95,40.67
MISC,6.02,-17.46,84.78
ELECTRICAL,5.89,-7.75,67.13
AC,3.84,2.89,30.00
AUTO,3.51,-15.21,67.17


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2740833.0,22
AR,1191310.0,9
XR,1054077.0,13
X40,875806.0,10
X40N,564176.0,10
OX40N,450824.0,10
XY25,413830.0,7
SR,198513.0,2
X5K,128477.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2781682.0,28
M-SC,1403751.0,17
H-LC,1372793.0,15
H-MC,1156847.0,15
M-LC,410790.0,5
M-MC,319658.0,2
L-SC,242557.0,3
L-MC,59860.0,1
L-LC,40596.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,920942.0,6
M-SC,XY24,818717.0,7
H-SC,AR,659795.0,5
H-LC,X40,587396.0,5
H-SC,XR,501171.0,6
H-MC,XY24,462795.0,4
H-LC,AR,372077.0,2
M-MC,XY24,319658.0,2
H-SC,X40N,246757.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
